In [1]:
%load_ext autoreload

%autoreload 2
import imp
import factorilog


In [2]:
%pdb 

Automatic pdb calling has been turned ON


In [5]:


from factorilog import Wire, WireColor
import blueprint_layer as BlueprintLayer
import netlist_layer as NetlistLayer

with open("blueprints/sample.bp","r") as file:
    sample = file.read()
    c = BlueprintLayer.importBlueprint(sample)


In [6]:
# Simple test: Blueprint->Layout-> Netlist->Layout->Blueprint->Layout->Netlist. Compare netlist

layout1 = BlueprintLayer.importBlueprint(sample)
netlist1 = NetlistLayer.exportNetlist(layout1, meta=True)
layout2 = NetlistLayer.importNetlist(netlist1)
bp2 = BlueprintLayer.exportBlueprint(layout2)
layout3 = BlueprintLayer.importBlueprint(bp2)
netlist2 = NetlistLayer.exportNetlist(layout3, meta=True)

print(netlist1==netlist2)
print(netlist1)
print("\n")
print(netlist2)


False
c: medium-electric-pole        | 1
a <= c: 1 black if green > 2   | 2
b <= c: red = green + 0        | 3
e <= a, d: @ all if black = 0  | 4
e <= a, b: @ red if black = 1  | 5
e: medium-electric-pole        | 6
d <=: 5 red                    | 7
||
1 | -5 -2 
2 | -2.5 -2 S
3 | -2.5 0 S
4 | 0.5 0 S
5 | 0.5 -1 S
6 | 4 -1 
7 | -3 1 S
a | green 2o-5i 4i-5i
b | red 5i-3o
c | red 1-3i 2i-1
d | red 7-4i
e | green 6-5o 4o-6


a: medium-electric-pole        | 1
b <= a: 1 black if green > 2   | 2
e <= a: red = green + 0        | 3
c <= d, b: @ all if black = 0  | 4
c <= b, e: @ red if black = 1  | 5
c: medium-electric-pole        | 6
d <=: 5 red                    | 7
||
1 | -5 -2 
2 | -2.5 -2 S
3 | -2.5 0 S
4 | 0.5 0 S
5 | 0.5 -1 S
6 | 4 -1 
7 | -3 1 S
a | red 3i-1 2i-1
b | green 5i-4i 5i-2o
c | green 4o-6 6-5o
d | red 7-4i
e | red 3o-5i


In [ ]:
netlist_meta = c.toNetString(meta=True)
print(netlist_meta)

In [16]:
layout = factorilog.Layout.fromNetlist(netlist_meta)

{
  "Entities": [
    {
      "Descriptor": {
        "name": "medium-electric-pole"
      },
      "ID": 1,
      "nets": {
        "pass": [
          "a"
        ],
        "out": null
      }
    },
    {
      "Descriptor": {
        "name": "decider-combinator",
        "behavior": {
          "decider_conditions": {
            "output_signal": {
              "name": "signal-black",
              "type": "virtual"
            },
            "constant": 2,
            "first_signal": {
              "name": "signal-green",
              "type": "virtual"
            },
            "copy_count_from_input": false,
            "comparator": ">"
          }
        }
      },
      "ID": 2,
      "nets": {
        "out": [
          "e"
        ],
        "in": [
          "a"
        ]
      }
    },
    {
      "Descriptor": {
        "name": "arithmetic-combinator",
        "behavior": {
          "arithmetic_conditions": {
            "output_signal": {
              "name": "si

In [17]:
print(layout.toNetString(meta=True))

a: medium-electric-pole        | 1
e <= a: 1 black if green > 2   | 2
b <= a: red = green + 0        | 3
c <= e, d: @ all if black = 0  | 4
c <= e, b: @ red if black = 1  | 5
c: medium-electric-pole        | 6
d <=: 5 red                    | 7
||
1 | -5 -2 
2 | -2.5 -2 S
3 | -2.5 0 S
4 | 0.5 0 S
5 | 0.5 -1 S
6 | 4 -1 
7 | -3 1 S
e | green 5i-2o 5i-4i
c | green 5o-6 4o-6
d | red 4i-7
b | red 5i-3o
a | red 1-2i 3i-1


In [18]:
from slpp import slpp as lua
bp=lua.encode(layout.exportBlueprint())
#print(bp.replace("  "," ").replace("\t","").replace("\n",""))
print(bp.replace("  "," ").replace("\t","  "))

{
  {
    ["entity_number"] = 1,
    ["connections"] =   {
      ["1"] =     {
        ["red"] =       {
                  {
            ["circuit_id"] = 1,
            ["entity_id"] = 2
          },
                  {
            ["circuit_id"] = 1,
            ["entity_id"] = 3
          }
        }
      }
    },
    ["name"] = "medium-electric-pole",
    ["position"] =   {
      ["y"] = -2,
      ["x"] = -5
    }
  },
  {
    ["entity_number"] = 2,
    ["name"] = "decider-combinator",
    ["direction"] = 2,
    ["control_behavior"] =   {
      ["decider_conditions"] =     {
        ["output_signal"] =       {
          ["name"] = "signal-black",
          ["type"] = "virtual"
        },
        ["constant"] = 2,
        ["first_signal"] =       {
          ["name"] = "signal-green",
          ["type"] = "virtual"
        },
        ["copy_count_from_input"] = false,
        ["comparator"] = ">"
      }
    },
    ["connections"] =   {
      ["1"] =     {
        ["red"] =       {


In [19]:
#wires per term
l_ents = list(c.entities)
for ent in l_ents:
    print(ent.name,len(ent.terminals))
    for term in ent.terminals:
        print("\t",len(term.wires))
        

arithmetic-combinator 2
	 1
	 1
medium-electric-pole 1
	 2
decider-combinator 2
	 2
	 1
constant-combinator 1
	 1
medium-electric-pole 1
	 2
decider-combinator 2
	 3
	 1
decider-combinator 2
	 1
	 1


In [20]:
ar = l_ents[1]
con = c.getConnectedTerminals(ar.terminals[0],WireColor.red)
print(con)
print(list(con)[0].ent.name)


{<factorilog.Terminal object at 0x7f1159d72128>}
medium-electric-pole


In [21]:
net = c.getNetlist()
print(len(net))
for hyp in list(net):
    print(hyp.color, len(hyp.terminals))

5
WireColor.red 3
WireColor.red 2
WireColor.green 3
WireColor.red 2
WireColor.green 3


In [22]:
w=l_ents[1].terminals[0].wires
for w1 in l_ents[1].terminals[0].wires:
    print(w1.color)
    print(w1.terminals)

WireColor.green
frozenset({<factorilog.Terminal object at 0x7f2a64029550>, <factorilog.Terminal object at 0x7f2a640295f8>})
WireColor.red
frozenset({<factorilog.Terminal object at 0x7f2a64029710>, <factorilog.Terminal object at 0x7f2a64029550>})


In [23]:
#num wires
wires = set()
for ent in list(c.ents):
    for term in ent.terminals:
        wires.update(term.wires)
print(len(wires))

8


In [26]:

import pprint
from slpp import slpp as lua
pp = pprint.PrettyPrinter(indent=2)
pp.pprint(lua.decode(sample))

[ { 'connections': { '1': { 'red': [ {'circuit_id': 1, 'entity_id': 2},
                                     {'circuit_id': 1, 'entity_id': 3}]}},
    'entity_number': 1,
    'name': 'medium-electric-pole',
    'position': {'x': -5, 'y': -2}},
  { 'connections': { '1': {'red': [{'entity_id': 1}]},
                     '2': {'green': [{'circuit_id': 1, 'entity_id': 5}]}},
    'control_behavior': { 'decider_conditions': { 'comparator': '>',
                                                  'constant': 2,
                                                  'copy_count_from_input': False,
                                                  'first_signal': { 'name': 'signal-green',
                                                                    'type': 'virtual'},
                                                  'output_signal': { 'name': 'signal-black',
                                                                     'type': 'virtual'}}},
    'direction': 2,
    'entity_number': 2,
  

In [284]:
import test_parser
import json
imp.reload(test_parser)

text=\
"""84"""

parser = test_parser.TestParser(parseinfo=False)
ast = parser.parse(text, rule_name='start')
print(ast)
print(json.dumps(ast, indent=2)) # ASTs are JSON-friendy

['84']
[
  "84"
]
